## AI Grading of Module 2, Assignment 3

In [2]:
# Configure Sagemaker Image
# # Install dotenv
# %pip install python-dotenv
# # Upgrade langchain
# %pip install --upgrade langchain
# # Install anthropic support
# %pip install langchain-anthropic
# # Install openai support
# %pip install langchain-openai
# # Install langchain hub
# %pip install langchainhub
# # Uninstall pydantic
# %pip uninstall pydantic -y
# # Re install pydantic specific version of pydantic (original version ==1.10.14)
# %pip install pydantic==1.10.14

In [3]:
from dotenv import load_dotenv
import os
import langchain
import os
import zipfile
from bs4 import BeautifulSoup

In [4]:
# Load environment variables from .env file
load_dotenv()
# Now you can access the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

### Setup the prompt

In [6]:
# This is the description of the assignment for the students
instructions = '''
The instructional material so far in this class has included:
    Prompt Engineering, The Golden Rule of Prompt Enginineering and large language model output evaluation
    Using Python code to interact with large language models
    Making direct API calls to LLMS using the requests Python package
    Using HuggingFace open source models to perform specific NLP tasks.
    Using prompt templates, large language Models and output parsers to create and execute workflow chains

Your assignment is to consider these lessons and describe a project of you own design which implements one or more of these topics.

You should pick a goal to achieve and descrete steps needed to achieve that goal.

Suggestions:
    Pick a goal that interests you.
    Do some research on the type of data needed for your project which you can access and what format it takes.
    Try to use one or more of the AI tools we've covered to achieve your goal, but your focus should be on achieving your goal, not on using the tools.

Your task is to propose, describe and document steps needed to achieve your goal, but not implement it in Python.  

Submitting Python code is optional.

Please describe your project, including the steps you will take in the project.
Think logicaly about your goal and the steps needed to progress from step one to the final step achieving your goal.
'''

In [12]:
# This is the grading criteria for the assignment
grading_rubric = '''
Evaluate the student's submission based on this list of prioritized criteria:
1. Does the student fully complete the assignment description?
2. Does the student state the goal clearly and unambiguously so that a classmate could understand?
3. Does the student list steps to achieve the goal in the logical order that progresses from data gathering through achieving the goal?
4. Does the student use one or more of the instructional topics covered in class?

After evaluating these prioritized criteria, assign a point value according to this scale:

0 points: did not submit the assignment
1 point: Student submission only met 1 or 2 of the prioritized criteria.
2 points: Student submission met 3 of the prioritized criteria, but the description was hard to understand.
3 points: Student submission met 4 of the prioritized criteria and was moderately easy to understand.
4 points: Student submission met all of the prioritized criteria, but didn't provide logical workflow steps to achieve the goal.
5 points: Student submission fully addressed the prioritized criteria, was written very clearly, provided a logical set of steps to achieve the goal and incorporated all parts of the instructional topics covered in class.

Use this format for your reponse. Please follow this format exactly:
Start with the student's name on the first line.
Next, give a very brief summary of the project, including the logical steps that were submitted.
Next, on  separate line, give the score of 0-5.
Next, explain any reasons the student did not get full credit.
Finally, make suggestions on how the student's project can be improved to achieve their stated goal.

'''

In [13]:
# I used this to test my prompt before I ran the loop on the whole class.
# This is where I pasted the student's assignment submission
data = '''

'''

In [14]:
# Use the ChatPromptTemplate
#
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are teaching assistant tasked with helping to grade and assignment. You are an expert in evaluating adherence to following directions."),
    ("human", "{assignment}"),
    ("ai", "Yes, I fully understand the assignment now, please give me the grading criteria."),
    ("human", "{grading_criteria}"),
    ("ai", "Yes, I fully understand the assignment and grading criteria now, please give me the student's submission and I will evaluate the submission according to the assignment description and grading criteria and give a response consistent with your instructions."),
    ("human", "{student_submission}"),
])
# What do we need to provde to use it? 
template.input_schema()

PromptInput(assignment=None, grading_criteria=None, student_submission=None)

In [15]:
# Test it out if you want to see an example prompt.
# You must provide content on the data variable.
#template.invoke({'assignment': instructions, 'grading_criteria': grading_rubric, 'student_submission':data})

In [16]:
# Create chain with this prompt, a model and a parser
# ChatGPT model
from langchain_openai import ChatOpenAI
openai_llm = ChatOpenAI(model='gpt-3.5-turbo', api_key=openai_api_key, temperature = 0)
# Could also use Claude here
#from langchain_anthropic import ChatAnthropic
#claude_llm = ChatAnthropic(model="claude-3-sonnet-20240229",api_key=anthropic_api_key, temperature = 0)
#
# This parser takes the AIMessage reutrned form the LLM and converts it to a string
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
#
# Create chain for OpenAI model
chain = template | openai_llm | output_parser

In [20]:
# I used this to test it out before running the full loop on the class submissions
#response = chain.invoke({'assignment': instructions, 'grading_criteria': grading_rubric, 'student_submission':data})

In [10]:
#print(response)

### Load the Assignment #3 data submitted by the students

In [11]:
import zipfile

# Extract Canvas downloaded zip file containing the students' submissions
with zipfile.ZipFile('./submissions.zip', 'r') as zip_ref:
    # Extract all the contents of the zip file to specified directory
    zip_ref.extractall("./assignment3")

In [18]:
# Get a list of filenames to evaluate, only those with .html extension are the submissions

assignment3_filenames = [f for f in os.listdir('./assignment3') if f.endswith('.html')]
# Iterate through the file names (used for testing
#for fn in assignment3_filenames:
    #print(fn)
    #pass

### This is the loop that evaluates all the submissions and saves the response to a text file

In [19]:
for fn in assignment3_filenames:
    #Open the HTML file and read its contents
    with open('./assignment3/' + fn, 'r') as f:
        print('Filename:', fn)
        # extract just the user
        user = fn.split("_")[0]
        print('User:', user)
        # Read the entire raw html file
        html_content = f.read()
        # Create a new  BeautifulSoup HTML parser
        soup = BeautifulSoup(html_content, 'html.parser')
        # Extract just the text using the parser (strip all the HTML tags)
        data = soup.get_text()
        # Extract just the student's name in the first line
        name = data.split(':')[1]
        #print(name,user,data)
        #print(data)
        # Invoke the chain
        response = chain.invoke({'assignment': instructions, 'grading_criteria': grading_rubric, 'student_submission':data})
        # Save the response to a file.
        new_fn = user + '.txt'
        with open('assignment3/grades/' + new_fn, 'a') as file:
            file.write(response)
        print('Wrote file: assignment3/grades/', new_fn)

Filename: shahprit_47824_text.html
User: shahprit
Wrote file: assignment3/grades/ shahprit.txt
Filename: mmoattarimehrpad_LATE_47276_text.html
User: mmoattarimehrpad
Wrote file: assignment3/grades/ mmoattarimehrpad.txt
Filename: upadhyayapranav_100184_text.html
User: upadhyayapranav
Wrote file: assignment3/grades/ upadhyayapranav.txt
Filename: kingseanpatrick_96812_text.html
User: kingseanpatrick
Wrote file: assignment3/grades/ kingseanpatrick.txt
Filename: sakodakayleechiyoko_97056_text.html
User: sakodakayleechiyoko
Wrote file: assignment3/grades/ sakodakayleechiyoko.txt
Filename: yamashitahinano_17223_text.html
User: yamashitahinano
Wrote file: assignment3/grades/ yamashitahinano.txt
Filename: buckleykadenrand_10012_text.html
User: buckleykadenrand
Wrote file: assignment3/grades/ buckleykadenrand.txt
Filename: chanted_21336_text.html
User: chanted
Wrote file: assignment3/grades/ chanted.txt
Filename: sheehanolivermichael_26838_text.html
User: sheehanolivermichael
Wrote file: assignm